In [ ]:
import cv2
import numpy as np
import joblib
import pygame  # For playing the alarm sound
import threading
import time
import tkinter as tk
from tkinter import messagebox
from twilio.rest import Client

# Twilio Credentials (Replace with actual credentials)
TWILIO_SID = "Your Twilio credentials"
TWILIO_AUTH_TOKEN = "Your Twilio credentials"
TWILIO_WHATSAPP_NUMBER = "whatsapp:+14155238886"  # Twilio Sandbox Number
TO_WHATSAPP_NUMBER = "Emergency Whatsapp Number"  # Replace with your WhatsApp number

client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)

def send_whatsapp_alert():
    message = client.messages.create(
        body="\u26A0\uFE0F Drowsiness Alert! The driver might be drowsy. Please check immediately.",
        from_=TWILIO_WHATSAPP_NUMBER,
        to=TO_WHATSAPP_NUMBER
    )
    print(f"WhatsApp Alert Sent: {message.sid}")

# Load the trained model
clf = joblib.load('drowsiness_detector_model.pkl')

# Load Haarcascades
face_cascade = cv2.CascadeClassifier('C:/Users/Lenovo/Downloads/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('C:/Users/Lenovo/Downloads/haarcascade_eye.xml')
mouth_cascade = cv2.CascadeClassifier('C:/Users/Lenovo/Downloads/haarcascade_mcs_mouth.xml')

# Initialize alarm
pygame.mixer.init()
alarm_sound = "H:\\mixkit-critical-alarm-1004.wav"  # Update this with your .wav file path

def play_alarm():
    pygame.mixer.music.load(alarm_sound)
    pygame.mixer.music.play()

def start_detection():
    global running
    running = True
    cap = cv2.VideoCapture(0)
    drowsiness_counter = 0
    frame_skip = 5 # Process every 10th frame to reduce lag
    frame_count = 0
    eye_closed_time = 0
    alarm_triggered = False

    while running:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % frame_skip != 0:
            continue  # Skip frames for better performance

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        drowsy_detected = False
        for (x, y, w, h) in faces:
            face_roi = gray[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(face_roi, scaleFactor=1.2, minNeighbors=6, minSize=(30, 30))
            mouth = mouth_cascade.detectMultiScale(face_roi, scaleFactor=1.5, minNeighbors=15, minSize=(30, 30))

            eyes_closed = len(eyes) == 0
            yawning = len(mouth) > 0

            if eyes_closed:
                if eye_closed_time == 0:
                    eye_closed_time = time.time()  # Start timer when eyes are closed
                elif time.time() - eye_closed_time >= 1:  # If eyes closed for 1 second
                    drowsy_detected = True
            else:
                eye_closed_time = 0  # Reset timer if eyes are open

            if yawning:
                drowsy_detected = True

            if drowsy_detected:
                drowsiness_counter += 1
            else:
                drowsiness_counter = 0  # Reset counter if awake

            if drowsiness_counter >= 4 or drowsy_detected:  # Alert conditions
                cv2.putText(frame, "DROWSY!", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                if not alarm_triggered:
                    threading.Thread(target=play_alarm, daemon=True).start()  # Play alarm sound
                    threading.Thread(target=send_whatsapp_alert, daemon=True).start()  # Send WhatsApp alert
                    alarm_triggered = True
            else:
                cv2.putText(frame, "Awake", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                alarm_triggered = False  # Reset alarm trigger if not drowsy

        cv2.imshow("Drowsiness Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

def stop_detection():
    global running
    running = False
    messagebox.showinfo("Drowsiness Detector", "Detection Stopped!")

# Tkinter GUI
root = tk.Tk()
root.title("Drowsiness Detector")

start_button = tk.Button(root, text="Start Detection", command=lambda: threading.Thread(target=start_detection).start(), height=2, width=20)
start_button.pack(pady=10)

stop_button = tk.Button(root, text="Stop Detection", command=stop_detection, height=2, width=20)
stop_button.pack(pady=10)

root.mainloop()

pygame 2.6.1 (SDL 2.28.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 716, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 404, in _make_request
    self._validate_conn(conn)
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1061, in _validate_conn
    conn.connect()
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connection.py", line 419, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 458, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 502, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\Lenovo\anaconda3\lib\ssl.py", line 501, in wrap_socke

Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 716, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 404, in _make_request
    self._validate_conn(conn)
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1061, in _validate_conn
    conn.connect()
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connection.py", line 419, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 458, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 502, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\Lenovo\anaconda3\lib\ssl.py", line 501, in wrap_sock

Exception in thread Thread-15:
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 716, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 404, in _make_request
    self._validate_conn(conn)
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1061, in _validate_conn
    conn.connect()
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connection.py", line 419, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 458, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 502, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\Lenovo\anaconda3\lib\ssl.py", line 501, in wrap_sock

Exception in thread Thread-19:
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 716, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 404, in _make_request
    self._validate_conn(conn)
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1061, in _validate_conn
    conn.connect()
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connection.py", line 419, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 458, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 502, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\Lenovo\anaconda3\lib\ssl.py", line 501, in wrap_sock

...

Exception in thread Thread-39:
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 716, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 404, in _make_request
    self._validate_conn(conn)
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1061, in _validate_conn
    conn.connect()
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connection.py", line 419, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 458, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 502, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\Lenovo\anaconda3\lib\ssl.py", line 501, in wrap_sock

Exception in thread Thread-43:
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 716, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 404, in _make_request
    self._validate_conn(conn)
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1061, in _validate_conn
    conn.connect()
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connection.py", line 419, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 458, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 502, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\Lenovo\anaconda3\lib\ssl.py", line 501, in wrap_sock

Exception in thread Thread-47:
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 716, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 404, in _make_request
    self._validate_conn(conn)
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1061, in _validate_conn
    conn.connect()
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connection.py", line 419, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 458, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 502, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\Lenovo\anaconda3\lib\ssl.py", line 501, in wrap_sock

Exception in thread Thread-51:
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 716, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 404, in _make_request
    self._validate_conn(conn)
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1061, in _validate_conn
    conn.connect()
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connection.py", line 419, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 458, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 502, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\Lenovo\anaconda3\lib\ssl.py", line 501, in wrap_sock

Exception in thread Thread-55:
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 716, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 404, in _make_request
    self._validate_conn(conn)
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 1061, in _validate_conn
    conn.connect()
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\connection.py", line 419, in connect
    self.sock = ssl_wrap_socket(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 458, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "C:\Users\Lenovo\anaconda3\lib\site-packages\urllib3\util\ssl_.py", line 502, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_hostname=server_hostname)
  File "C:\Users\Lenovo\anaconda3\lib\ssl.py", line 501, in wrap_sock